Thinking1: 如何使用用户标签来指导业务（如何提升业务）

1. 精准营销。通过用户标签匹配用户更有可能感兴趣的商品或服务，获取新客户。
2. 个性化推荐。根据用户标签进行个性化推荐，让用户能够获取到更感兴趣的内容，提升用户使用时长。
3. 流失率预测。在关键节点，定向推送优惠信息，留存用户。

Thinking2:如果给你一堆用户数据，没有打标签。你该如何处理（如何打标签）

1. 请人工打标签。比较贵，但质量好。
2. 自动打标签。通过聚类，成本低，质量较差。

Thinking3: 准确率和精确率有何不同（评估指标）

- 准确率：$accuracy=\frac{TP+TN}{TP+FP+TN+FN}$， 所有预测正确的样本中占总样本的比例
- 精确率：$precision=\frac{TP}{TP+FP}$， 在所有正的样本中，预测正确的比例

Thinking4: 如果你使用大众点评，想要给某个餐厅打标签。这时系统可以自动提示一些标签，你会如何设计（标签推荐）

- 菜品相关：种类、口味、价位区间
- 服务水平：服务及时
- 就餐环境：是否可订座、有无包厢、有无车位、有无大桌

Thinking5: 我们今天使用了10种方式来解MNIST，这些方法有何不同？你还有其他方法来解决MNIST识别问题么（分类方法）

- Logistic Regression：根据每个像素点的权重来分类。
- Cart，ID3：树模型，会根据某一些重要的像素点进行分类。
- LDA：(Linear Discriminant Analysis)一种监督学习的降维技术。
- Naive Bayes:直接找出特征输出Y和特征X的联合分布。需要特征之间彼此独立。
- SVM: 最大化分类间隔，相比于LR，效果更好。
- KNN：找训练集中最匹配的K个样本作为分类依据。
- AdaBoost：集成学习，用boosting思想，自适应设置各弱分类器的权重。
- XGBoost：GBDT的高效实现，性能提升、精度提升，支持的弱学习器不仅局限于决策树
- TPOT：自动机器学习，需要遍历各种模型及参数，准确率高，但效率较低。
- keras：深度模型。图象识别一般用CNN，能够高效提取边缘等图象特征，如果加入anchor可解决大小、旋转敏感问题。

Action1：针对Delicious数据集，对SimpleTagBased算法进行改进（使用NormTagBased、TagBased-TFIDF算法）

In [1]:
import random
import math
import operator
import pandas as pd
from collections import defaultdict
import heapq

In [2]:
# 使用SimpleTagBased算法对Delicious2K数据进行推荐
# 原始数据集：https://grouplens.org/datasets/hetrec-2011/
# 数据格式：userID     bookmarkID     tagID     timestamp
file_path = "./user_taggedbookmarks-timestamps.dat"
# 字典类型，保存了user对item的tag，即{userid: {item1:[tag1, tag2], ...}}
records = defaultdict(lambda: defaultdict(list))        
# 数据加载
def load_data():
    print("开始数据加载...")
    df = pd.read_csv(file_path, sep='\t')
    for i in range(len(df)):
        uid = df['userID'][i]
        iid = df['bookmarkID'][i]
        tag = df['tagID'][i]      
        records[uid][iid].append(tag)
    print("数据集大小为 %d." % (len(df)))
    print("设置tag的人数 %d." % (len(records)))
    print("数据加载完成\n")

# 将数据集拆分为训练集和测试集
# 训练集，测试集
train_data = defaultdict(lambda: defaultdict(list))
test_data = defaultdict(lambda: defaultdict(list))
def train_test_split(ratio, seed=100):
    random.seed(seed)
    for u in records.keys():
        for i in records[u].keys():
            # ratio比例设置为测试集
            if random.random()<ratio:
                for t in records[u][i]:
                    test_data[u][i].append(t)
            else:
                for t in records[u][i]:
                    train_data[u][i].append(t)        
    print("训练集样本数 %d, 测试集样本数 %d" % (len(train_data),len(test_data)))

# 数据加载
load_data()
# 训练集，测试集拆分，20%测试集
train_test_split(0.2)

开始数据加载...
数据集大小为 437593.
设置tag的人数 1867.
数据加载完成

训练集样本数 1860, 测试集样本数 1793


In [11]:
# 使用训练集，初始化user_tags, tag_items, user_items
# 用户标签，商品标签
user_tags = defaultdict(lambda: defaultdict(int))
tag_items = defaultdict(lambda: defaultdict(int))
user_items = defaultdict(lambda: defaultdict(int))

item_users = defaultdict(lambda: defaultdict(int))
item_tags = defaultdict(lambda: defaultdict(int))
tag_users = defaultdict(lambda: defaultdict(int))
def initStat():
    records=train_data
    for u,items in records.items():
        for i,tags in items.items():
            for tag in tags:
                #print tag
                # 用户和tag的关系
                user_tags[u][tag] += 1
                # tag和item的关系
                tag_items[tag][i] += 1
                # 用户和item的关系
                user_items[u][i] += 1
                # tag和user的关系
                tag_users[tag][u] += 1
                # item和tag的关系
                item_tags[i][tag] += 1
                # item和user的关系
                item_users[i][u] += 1
    print("user_tags, tag_items, user_items初始化完成.")
    print("user_tags大小 %d, tag_items大小 %d, user_items大小 %d" % (len(user_tags), len(tag_items), len(user_items)))
    print("tag_users大小 %d, item_tags大小 %d, item_users大小 %d" % (len(tag_users), len(item_tags), len(item_users)))
    
initStat()

user_tags, tag_items, user_items初始化完成.
user_tags大小 1860, tag_items大小 36884, user_items大小 1860
tag_users大小 36884, item_tags大小 59555, item_users大小 59555


In [4]:
# 对用户user推荐Top-N
def recommend(user, N):
    recommend_items=defaultdict(int)
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    tagged_items = user_items[user]   
    for tag, wut in user_tags[user].items():
        #print(self.user_tags[user].items())
        for item, wti in tag_items[tag].items():
            if item in tagged_items:
                continue
            #print('wut = %s, wti = %s' %(wut, wti))
            recommend_items[item] += wut * wti / norm(user, tag)
    return heapq.nlargest(N, recommend_items.items(), key=operator.itemgetter(1))

In [5]:
# 使用测试集，计算准确率和召回率
def precisionAndRecall(N):
    hit = 0
    h_recall = 0
    h_precision = 0
    for user,items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        rank = recommend(user, N)
        for item,rui in rank:
            if item in items:
                hit = hit + 1
        h_recall += len(items)
        h_precision += N
    #print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
    # 返回准确率 和 召回率
    return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))


In [6]:
# 使用测试集，对推荐结果进行评估
def testRecommend():
    print("推荐结果评估")
    print("%3s %10s %10s" % ('N',"精确率",'召回率'))
    for n in [5,10,20,40,60,80,100]:
        precision,recall = precisionAndRecall(n)
        print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))

In [7]:
# SimpleTagBased
def norm(user, tag):
    return 1  
testRecommend()

推荐结果评估
  N        精确率        召回率
  5      0.829%      0.355%
 10      0.633%      0.542%
 20      0.512%      0.877%
 40      0.381%      1.304%
 60      0.318%      1.635%
 80      0.276%      1.893%
100      0.248%      2.124%


In [8]:
# NormTagBased-1
def norm(user, tag):
    return len(user_tags[user].items())*len(tag_users[tag].items())
testRecommend()

推荐结果评估
  N        精确率        召回率
  5      0.907%      0.388%
 10      0.638%      0.546%
 20      0.507%      0.868%
 40      0.356%      1.218%
 60      0.287%      1.476%
 80      0.255%      1.750%
100      0.241%      2.061%


In [9]:
# NormTagBased-2
def norm(user, tag):
    return len(user_tags[user].items())*len(tag_items[tag].items())
testRecommend()

推荐结果评估
  N        精确率        召回率
  5      0.806%      0.345%
 10      0.577%      0.494%
 20      0.428%      0.733%
 40      0.300%      1.026%
 60      0.259%      1.333%
 80      0.237%      1.620%
100      0.222%      1.903%


In [10]:
# TagBased-TFIDF
from math import log
def norm(user, tag):
    return log(1+len(tag_users[tag].items()))
testRecommend()

推荐结果评估
  N        精确率        召回率
  5      1.008%      0.431%
 10      0.761%      0.652%
 20      0.549%      0.940%
 40      0.402%      1.376%
 60      0.328%      1.687%
 80      0.297%      2.033%
100      0.269%      2.306%


Action2: 对Titanic数据进行清洗，建模并对乘客生存进行预测。使用之前介绍过的10种模型中的至少2种（包括TPOT）

- 数据清洗

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
# 读取数据
train_data = pd.read_csv('./Titanic_Data-master/train.csv')
test_data = pd.read_csv('./Titanic_Data-master/test.csv')

In [3]:
#  Age的缺失值用平均值填充
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)
# Embarked:训练集中有两个缺失值用众数填充，测试集没有缺失
embarked_count = train_data['Embarked'].value_counts()
train_data['Embarked'].fillna('S', inplace=True)
# Fare：测试集中有一个缺失值，用平均值填充
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)
# Cabin缺失值太多直接舍弃
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
test_features = test_data[features]
train_labels = train_data['Survived']
# 进行one-hot编码
dvec = DictVectorizer(sparse=False)
train_features = dvec.fit_transform(train_features.to_dict(orient='records'))
test_features = dvec.transform(test_features.to_dict(orient='records'))
# 训练集验证集切割
train_x, test_x, train_y, test_y = train_test_split(
    train_features, train_labels, test_size=0.3, random_state=42)

# 采用Z-Score规范化，有些模型需要归一化输入
ss = StandardScaler()
train_ss_features = ss.fit_transform(train_features)
train_ss_x = ss.transform(train_x)
test_ss_x = ss.transform(test_x)

- TPOT自动机器学习

In [10]:
# TPOT自动机器学习
from tpot import TPOTClassifier
# tpot采用遗传算法，generations：遗传几代；population_size：每代数目。
pipeline_optimizer = TPOTClassifier(generations=50, population_size=20, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(train_features, train_labels)
# 最终得到的最好的模型是随机森林，基分类器是线性SVM，参数如下：
# RandomForestClassifier(
#     StandardScaler(LinearSVC(input_matrix, C=0.001, dual=True, loss=squared_hinge, penalty=l2, tol=0.0001)), 
#     bootstrap=False, criterion=entropy, max_features=0.55, 
#     min_samples_leaf=5, min_samples_split=4, n_estimators=100)
# cv准确率0.844
pipeline_optimizer.export('tpot_model.py')


Generation 1 - Current best internal CV score: 0.8316615403929447

Generation 2 - Current best internal CV score: 0.8316615403929447

Generation 3 - Current best internal CV score: 0.8339087314041805

Generation 4 - Current best internal CV score: 0.8339087314041805

Generation 5 - Current best internal CV score: 0.8350260498399347

Generation 6 - Current best internal CV score: 0.8350260498399347

Generation 7 - Current best internal CV score: 0.8350260498399347

Generation 8 - Current best internal CV score: 0.8428912183792606

Generation 9 - Current best internal CV score: 0.8428912183792606

Generation 10 - Current best internal CV score: 0.8428912183792606

Generation 11 - Current best internal CV score: 0.8428912183792606

Generation 12 - Current best internal CV score: 0.8428912183792606

Generation 13 - Current best internal CV score: 0.8428912183792606

Generation 14 - Current best internal CV score: 0.8428912183792606

Generation 15 - Current best internal CV score: 0.842891

TPOTClassifier(generations=50, population_size=20, random_state=42, verbosity=2)

- 决策树模型

In [20]:
# 决策树模型
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(train_x, train_y)
pred_labels_cart = dt.predict(test_features)
print('决策树模型交叉验证准确率：', np.mean(cross_val_score(dt, train_features, train_labels, cv=5)))

决策树模型交叉验证准确率： 0.7766932395957568


- XGBoost模型

In [70]:
from xgboost import XGBClassifier
bst = XGBClassifier(
        learning_rate=0.01,#学习率
        n_estimators=3000,#步长
        max_depth=4,#深度
        objective='binary:logistic',
        seed=27,
        eval_metric = 'auc',
        tree_mothod='hist'
    )
bst.fit(train_x,train_y)
#预测结果
pred_labels_xgb = bst.predict(test_x)
# 交叉验证准确率
print('XGBoost交叉验证准确率:', np.mean(cross_val_score(bst, train_features, train_labels, cv=5)))

XGBoost交叉验证准确率: 0.8305630531667818


- 逻辑回归模型

In [29]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_ss_x, train_y)
predict_labels_lr = lr.predict(test_ss_x)
# 交叉验证准确率
print('LR交叉验证准确率:', np.mean(cross_val_score(lr, train_ss_features, train_labels, cv=5)))

LR交叉验证准确率: 0.7856506182913816


- LDA分类器

In [31]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
lda = LinearDiscriminantAnalysis()
lda.fit(train_ss_x,train_y)
predict_labels_lda=lda.predict(test_ss_x)
# 交叉验证准确率
print('LDA交叉验证准确率:', np.mean(cross_val_score(lda, train_ss_features, train_labels, cv=5)))

LDA准确率: 0.7946142740568702


- 贝叶斯分类器

In [44]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
nb = GaussianNB()
nb.fit(train_x,train_y)
predict_labels_nb = nb.predict(test_x)
# 交叉验证准确率
print('朴素贝叶斯交叉验证准确率:',  np.mean(cross_val_score(nb, train_features, train_labels, cv=5)))

朴素贝叶斯交叉验证准确率: 0.7890276818780995


- SVM分类器

In [4]:
from sklearn import svm
svc = svm.SVC(kernel='rbf', C=1.0, gamma='auto')
svc.fit(train_ss_x,train_y)
predict_labels_svc=svc.predict(test_ss_x)
# 交叉验证准确率
print('SVM交叉验证准确率:',  np.mean(cross_val_score(svc, train_ss_features, train_labels, cv=5)))

SVM交叉验证准确率: 0.8249074132195091


- KNN分类器

In [38]:
from sklearn.neighbors import KNeighborsClassifier #KNN
knn = KNeighborsClassifier()
knn.fit(train_ss_x,train_y)
predict_labels_knn=knn.predict(test_ss_x)
# 交叉验证准确率
print('KNN交叉验证准确率:', np.mean(cross_val_score(knn, train_ss_features, train_labels, cv=5)))

KNN交叉验证准确率: 0.8036093151716777


- AdaBoost分类器

In [41]:
from sklearn.ensemble import  AdaBoostClassifier 
# 弱分类器
dt_stump = DecisionTreeClassifier(max_depth=5,min_samples_leaf=1)
dt_stump.fit(train_ss_x, train_y)
#dt_stump_err = 1.0-dt_stump.score(test_x, test_y)
# 设置AdaBoost迭代次数
n_estimators=500
abst = AdaBoostClassifier(base_estimator=dt_stump,n_estimators=n_estimators)
abst.fit(train_ss_x,train_y)
predict_labels_abst=abst.predict(test_ss_x)
# 交叉验证准确率
print('AdaBoost交叉验证准确率:',  np.mean(cross_val_score(abst, train_ss_features, train_labels, cv=5)))

AdaBoost交叉验证准确率: 0.8013684012303057
